In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df = pd.read_spss('CCAM SPSS Data 2008-2022.sav')
df.head()

,case_ID,wave,year,weight_wave,weight_aggregate,happening,cause_original,cause_other_text,cause_recoded,sci_consensus,...,employment,house_head,house_size,house_ages0to1,house_ages2to5,house_ages6to12,house_ages13to17,house_ages18plus,house_type,house_own
0,2.0,Nov 2008,2008,0.54,0.269626,Yes,Caused mostly by human activities,,Caused mostly by human activities,Most scientists think global warming is happening,...,Not working - retired,Not head of household,3.0,0.0,0.0,0.0,0.0,3.0,One-family house detached from any other house,Owned by you or someone in your household
1,3.0,Nov 2008,2008,0.85,0.424411,Don't know,Caused mostly by human activities,,Caused mostly by human activities,Don't know enough to say,...,Not working - disabled,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,Mobile home,Rented
2,5.0,Nov 2008,2008,0.49,0.244660,Don't know,Caused mostly by natural changes in the enviro...,,Caused mostly by natural changes in the enviro...,There is a lot of disagreement among scientist...,...,Not working - looking for work,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household
3,6.0,Nov 2008,2008,0.29,0.144799,Yes,Caused mostly by natural changes in the enviro...,,Caused mostly by natural changes in the enviro...,Most scientists think global warming is happening,...,Not working - retired,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household
4,7.0,Nov 2008,2008,1.29,0.644106,Yes,Caused mostly by human activities,,Caused mostly by human activities,There is a lot of disagreement among scientist...,...,Working - as a paid employee,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household


In [3]:
df.columns

Index(['case_ID', 'wave', 'year', 'weight_wave', 'weight_aggregate',
       'happening', 'cause_original', 'cause_other_text', 'cause_recoded',
       'sci_consensus', 'worry', 'harm_personally', 'harm_US',
       'harm_dev_countries', 'harm_future_gen', 'harm_plants_animals',
       'when_harm_US', 'reg_CO2_pollutant', 'reg_utilities', 'fund_research',
       'reg_coal_emissions', 'discuss_GW', 'hear_GW_media', 'gender', 'age',
       'age_category', 'generation', 'educ', 'educ_category', 'income',
       'income_category', 'race', 'ideology', 'party', 'party_w_leaners',
       'party_x_ideo', 'registered_voter', 'region9', 'region4', 'religion',
       'religion_other_nonchristian', 'evangelical', 'service_attendance',
       'marit_status', 'employment', 'house_head', 'house_size',
       'house_ages0to1', 'house_ages2to5', 'house_ages6to12',
       'house_ages13to17', 'house_ages18plus', 'house_type', 'house_own'],
      dtype='object')

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_text
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import plot_tree
from sklearn.inspection import permutation_importance

# Assuming your DataFrame is named 'df'
# Replace 'your_file.csv' with the actual file or use your DataFrame directly
# df = pd.read_csv('your_file.csv')

# Select relevant columns and split the data into features and target
features = ['harm_personally', 'harm_US', 'harm_dev_countries', 'harm_future_gen', 'harm_plants_animals',
            'reg_CO2_pollutant', 'reg_utilities', 'fund_research', 'reg_coal_emissions', 'discuss_GW',
            'hear_GW_media', 'gender', 'age_category', 'generation', 'educ_category', 'income_category',
            'race', 'ideology', 'party', 'party_w_leaners', 'party_x_ideo', 'registered_voter', 'region9',
            'region4', 'religion',  'evangelical', 'service_attendance',
            'marit_status', 'employment', 'house_head', 'house_size', 'house_ages0to1', 'house_ages2to5',
            'house_ages6to12', 'house_ages13to17', 'house_ages18plus', 'house_type', 'house_own']
standardize=['harm_personally', 'harm_US', 'harm_dev_countries', 'harm_future_gen', 'harm_plants_animals',
            'reg_CO2_pollutant', 'reg_utilities', 'fund_research', 'reg_coal_emissions', 'discuss_GW',
            'hear_GW_media', 'gender', 'age_category', 'generation', 'educ_category', 'income_category',
            'race', 'ideology', 'party', 'party_w_leaners', 'party_x_ideo', 'registered_voter', 'region9',
            'region4', 'religion',  'evangelical', 'service_attendance',
            'marit_status', 'employment', 'house_head', 'house_type', 'house_own']
df['evangelical'] = df['evangelical'].apply(lambda x: "Don't Know" if isinstance(x, float) else x)
target = 'happening'

# Loop through each year
for year in df['year'].unique():
    # Filter data for the current year
    subset = df[df['year'] == year]

    # Select features and target
    X = subset[features]
    y = subset[target]
    label_encoder = LabelEncoder()
    for column in standardize:
        X[column] = label_encoder.fit_transform(X[column])
    y=label_encoder.fit_transform(y)
    # Apply weights based on 'weight_aggregate'
    sample_weights = subset['weight_aggregate']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
        X, y, sample_weights, test_size=0.2, random_state=42
    )

    # Create a decision tree classifier
    model = DecisionTreeClassifier(max_depth=5)

    # Train the model using the training sets and weights
    model.fit(X_train, y_train, sample_weight=weights_train)

    # Make predictions on the testing set
    y_pred = model.predict(X_test)

    # Evaluate the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred, sample_weight=weights_test)
    print(f"Accuracy for year {year}: {accuracy * 100:.2f}%")
    


    # Display feature importances
    # Calculate permutation importances
    result = permutation_importance(model, X_test, y_test, n_repeats=30, random_state=42, n_jobs=-1)

    # Display feature importances
    feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': result.importances_mean})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
    print(f"Permutation Feature Importances for year {year}:\n{feature_importance_df}\n")



C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Accuracy for year 2008: 73.50%
Permutation Feature Importances for year 2008:
                Feature  Importance
4   harm_plants_animals    0.056967
3       harm_future_gen    0.035951
2    harm_dev_countries    0.007544
16                 race    0.005158
20         party_x_ideo    0.005004
24             religion    0.003079
23              region4    0.002309
21     registered_voter    0.001771
7         fund_research    0.000924
35     house_ages18plus    0.000616
15      income_category    0.000616
28           employment    0.000000
27         marit_status    0.000000
0       harm_personally    0.000000
29           house_head    0.000000
30           house_size    0.000000
31       house_ages0to1    0.000000
33      house_ages6to12    0.000000
34     house_ages13to17    0.000000
36           house_type    0.000000
32       house_ages2to5    0.000000
19      party_w_leaners    0.000000
1               harm_US    0.000000
18                party    0.000000
17             ideolog

C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Permutation Feature Importances for year 2010:
                Feature  Importance
4   harm_plants_animals    0.186584
2    harm_dev_countries    0.059095
0       harm_personally    0.034733
20         party_x_ideo    0.014979
16                 race    0.004774
32       house_ages2to5    0.003128
36           house_type    0.002305
22              region9    0.001975
35     house_ages18plus    0.001070
34     house_ages13to17    0.000000
31       house_ages0to1    0.000000
18                party    0.000000
29           house_head    0.000000
28           employment    0.000000
26   service_attendance    0.000000
25          evangelical    0.000000
23              region4    0.000000
30           house_size    0.000000
37            house_own    0.000000
11               gender    0.000000
10        hear_GW_media    0.000000
15      income_category    0.000000
14        educ_category    0.000000
3       harm_future_gen    0.000000
13           generation    0.000000
7         fund_re

C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

...

Permutation Feature Importances for year 2020:
                Feature  Importance
3       harm_future_gen    0.076917
9            discuss_GW    0.032849
2    harm_dev_countries    0.019774
20         party_x_ideo    0.016707
4   harm_plants_animals    0.010896
28           employment    0.004036
23              region4    0.003632
7         fund_research    0.002421
35     house_ages18plus    0.001372
36           house_type    0.001291
22              region9    0.000726
27         marit_status    0.000000
24             religion    0.000000
25          evangelical    0.000000
26   service_attendance    0.000000
0       harm_personally    0.000000
29           house_head    0.000000
30           house_size    0.000000
32       house_ages2to5    0.000000
34     house_ages13to17    0.000000
33      house_ages6to12    0.000000
19      party_w_leaners    0.000000
1               harm_US    0.000000
18                party    0.000000
17             ideology    0.000000
16               

C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Permutation Feature Importances for year 2021:
                Feature  Importance
3       harm_future_gen    0.084841
4   harm_plants_animals    0.056887
1               harm_US    0.049878
7         fund_research    0.009780
16                 race    0.007498
20         party_x_ideo    0.006112
14        educ_category    0.002608
12         age_category    0.002445
22              region9    0.002200
35     house_ages18plus    0.001956
37            house_own    0.001793
24             religion    0.000081
23              region4    0.000000
31       house_ages0to1    0.000000
28           employment    0.000000
32       house_ages2to5    0.000000
27         marit_status    0.000000
33      house_ages6to12    0.000000
26   service_attendance    0.000000
25          evangelical    0.000000
34     house_ages13to17    0.000000
36           house_type    0.000000
29           house_head    0.000000
0       harm_personally    0.000000
21     registered_voter    0.000000
18               

C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
C:\Users\echen\AppData\Local\Temp\ipykernel_25572\4262424432.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Permutation Feature Importances for year 2022:
                Feature  Importance
4   harm_plants_animals    0.121569
3       harm_future_gen    0.120752
1               harm_US    0.054085
20         party_x_ideo    0.026797
5     reg_CO2_pollutant    0.013889
17             ideology    0.005556
9            discuss_GW    0.001961
37            house_own    0.000327
33      house_ages6to12    0.000000
34     house_ages13to17    0.000000
35     house_ages18plus    0.000000
31       house_ages0to1    0.000000
29           house_head    0.000000
28           employment    0.000000
27         marit_status    0.000000
23              region4    0.000000
22              region9    0.000000
36           house_type    0.000000
21     registered_voter    0.000000
32       house_ages2to5    0.000000
0       harm_personally    0.000000
18                party    0.000000
16                 race    0.000000
2    harm_dev_countries    0.000000
6         reg_utilities    0.000000
7         fund_re

In [22]:
df[['harm_personally', 'harm_US', 'harm_dev_countries', 'harm_future_gen', 'harm_plants_animals',
            'reg_CO2_pollutant', 'reg_utilities', 'fund_research', 'reg_coal_emissions', 'discuss_GW',
            'hear_GW_media', 'gender', 'age_category', 'generation', 'educ_category', 'income_category',
            'race', 'ideology', 'party', 'party_w_leaners', 'party_x_ideo', 'registered_voter', 'region9',
            'region4', 'religion', 'religion_other_nonchristian', 'evangelical', 'service_attendance',
            'marit_status', 'employment', 'house_head', 'house_size', 'house_ages0to1', 'house_ages2to5',
            'house_ages6to12', 'house_ages13to17', 'house_ages18plus', 'house_type', 'house_own']]

,harm_personally,harm_US,harm_dev_countries,harm_future_gen,harm_plants_animals,reg_CO2_pollutant,reg_utilities,fund_research,reg_coal_emissions,discuss_GW,...,employment,house_head,house_size,house_ages0to1,house_ages2to5,house_ages6to12,house_ages13to17,house_ages18plus,house_type,house_own
0,Only a little,A moderate amount,A great deal,A great deal,A great deal,Strongly support,Strongly support,Strongly support,NaN,Occasionally,...,Not working - retired,Not head of household,3.0,0.0,0.0,0.0,0.0,3.0,One-family house detached from any other house,Owned by you or someone in your household
1,Only a little,Refused,Only a little,A moderate amount,A moderate amount,Somewhat support,Somewhat support,Somewhat support,NaN,Rarely,...,Not working - disabled,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,Mobile home,Rented
2,Not at all,Not at all,Not at all,Not at all,Not at all,Somewhat oppose,Strongly oppose,Strongly oppose,NaN,Never,...,Not working - looking for work,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household
3,Only a little,Only a little,A moderate amount,A moderate amount,A moderate amount,Somewhat support,Strongly support,Strongly support,NaN,Rarely,...,Not working - retired,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household
4,Don't know,Don't know,Don't know,Don't know,A moderate amount,Somewhat support,Strongly oppose,Strongly support,NaN,Never,...,Working - as a paid employee,Head of household,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30131,A moderate amount,A great deal,A great deal,A great deal,A great deal,Somewhat support,NaN,Strongly support,NaN,Rarely,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,2.0,NaN,Owned by you or someone in your household
30132,A moderate amount,Only a little,Only a little,Only a little,Only a little,Strongly oppose,NaN,Somewhat support,NaN,Never,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,Owned by you or someone in your household
30133,A moderate amount,A great deal,A great deal,A great deal,A great deal,Strongly support,NaN,Strongly support,NaN,Never,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2.0,NaN,Rented
30134,Not at all,Only a little,Not at all,Not at all,A moderate amount,Strongly oppose,NaN,Strongly oppose,NaN,Rarely,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,Owned by you or someone in your household


In [24]:
df[df['year']=='2021'].head()

,case_ID,wave,year,weight_wave,weight_aggregate,happening,cause_original,cause_other_text,cause_recoded,sci_consensus,...,employment,house_head,house_size,house_ages0to1,house_ages2to5,house_ages6to12,house_ages13to17,house_ages18plus,house_type,house_own
27075,48001.0,Mar 2021,2021,0.5832,0.607664,Don't know,Caused mostly by natural changes in the enviro...,,Caused mostly by natural changes in the enviro...,There is a lot of disagreement among scientist...,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,Owned by you or someone in your household
27076,48002.0,Mar 2021,2021,0.7256,0.756037,Yes,Caused mostly by human activities,,Caused mostly by human activities,There is a lot of disagreement among scientist...,...,NaN,NaN,8.0,NaN,NaN,NaN,NaN,7.0,NaN,Owned by you or someone in your household
27077,48003.0,Mar 2021,2021,0.7308,0.761456,Yes,Caused mostly by natural changes in the enviro...,,Caused mostly by natural changes in the enviro...,Don't know enough to say,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,Owned by you or someone in your household
27078,48004.0,Mar 2021,2021,0.5975,0.622564,No,None of the above because global warming isn't...,,Neither because global warming isn't happening,There is a lot of disagreement among scientist...,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,Owned by you or someone in your household
27079,48005.0,Mar 2021,2021,0.6942,0.723320,No,Caused mostly by natural changes in the enviro...,,Caused mostly by natural changes in the enviro...,Most scientists think global warming is not ha...,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,Owned by you or someone in your household


In [25]:
data_types = df[df['year']=='2021'].dtypes

# Identify columns with string values
string_columns = data_types[data_types == 'object'].index.tolist()

# Identify columns with float values
float_columns = data_types[data_types == 'float64'].index.tolist()

print("Columns with string values:", string_columns)
print("Columns with float values:", float_columns)

Columns with string values: ['cause_other_text', 'religion_other_nonchristian']
Columns with float values: ['case_ID', 'weight_wave', 'weight_aggregate', 'age', 'house_size', 'house_ages0to1', 'house_ages2to5', 'house_ages6to12', 'house_ages13to17', 'house_ages18plus']


In [28]:
df[df['year']=='2021']['evangelical'].dtype

CategoricalDtype(categories=[98.0, 'Don't Know', 'No', 'Refused', 'Yes'], ordered=False, categories_dtype=object)